<a href="https://colab.research.google.com/github/HwangHanJae/Book-Recommendation/blob/main/User_Base_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 로드

In [1]:
import pandas as pd
import numpy as np

rating_user_path = '/content/drive/MyDrive/Colab Notebooks/Recommendation/Book Recommendation/dataset/Rating_Users.csv'
rating_book_path = '/content/drive/MyDrive/Colab Notebooks/Recommendation/Book Recommendation/dataset/Rating_Books.csv'
new_ratings_path = '/content/drive/MyDrive/Colab Notebooks/Recommendation/Book Recommendation/dataset/New_Ratings.csv'

rating_users = pd.read_csv(rating_user_path)
rating_books = pd.read_csv(rating_book_path)
new_ratings = pd.read_csv(new_ratings_path)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 모델링

In [2]:
rating_matrix = {}
for i in new_ratings.index:
  user = new_ratings.loc[i, 'User-ID']
  isbn = new_ratings.loc[i, 'ISBN']
  rating = new_ratings.loc[i, 'Book-Rating']
  if user not in rating_matrix:
    rating_matrix[user] = {}
  rating_matrix[user][isbn] = rating

In [3]:
class Model():
  def __init__(self, matrix, users, items, num_sim_user=20):
    self.R = matrix
    self.users = set(users)
    self.items = set(items)
    self.num_users = len(users)
    self.num_items = len(items)
    self.num_sim_user = num_sim_user
    self._get_user_mean()
  def _get_Ixy(self, x, y):
    Ixy = set(self.R[x].keys() & self.R[y].keys())
    
    return Ixy
  
  def _get_I(self, u):
    I = set(self.R[u].keys())

    return I
  
  def _simil(self, x, y):
    Ixy = self._get_Ixy(x, y)
    Ix = self._get_I(x)
    Iy = self._get_I(y)
    rxi2s = []
    ryi2s = []
    numerator = []

    for i in Ixy:
      rxi = self.R[x][i]
      ryi = self.R[y][i]
      numerator.append((rxi*ryi))
    for i in Ix:
      rxi = self.R[x][i]
      rxi2s.append(rxi**2)
    for i in Iy:
      ryi = self.R[y][i]
      ryi2s.append(ryi**2)
   
    numerator = sum(numerator)
    denominator = (sum(rxi2s) **0.5) * (sum(ryi2s) ** 0.5)

    if denominator == 0:
      return 0
    else:
      return (numerator / denominator)
  
  def _get_user_sims(self, query_user):
    user_sim = {}
    for user in self.users - {query_user}:
      user_sim[user] = self._simil(query_user, user)
    
    return sorted(user_sim.items(), key=lambda x: x[1], reverse=True)
  def _get_user_mean(self):
    self.mean = {}
    for u, rating in self.R.items():
      self.mean[u] = sum(rating.values()) / len(rating)

  def recommend(self, user_id, num_recom_items=10):
    recommend_items = []

    print("피어그룹 계산 중...")     
    peergroup = self._get_user_sims(user_id)[:self.num_sim_user]
    print('피어그룹 계산 완료')
    recommend_items = []
    for item in self.items - self.R[user_id].keys():
      sum_k = 0
      sum_u = 0

      for user, simil in peergroup:
        if item not in self.R[user]:
          continue
        sum_u += simil * (self.R[user][item] - self.mean[user])
        sum_k += abs(simil)
      rui = self.mean[user_id] + (1 / sum_k) * sum_u if sum_k > 0.0 else 0.0
      recommend_items.append((item, rui))
    recommend_items.sort(key=lambda x : x[1], reverse=True)
    print('추천 완료')
    return [item for item, rui in recommend_items[:num_recom_items]]

In [4]:
users = new_ratings['User-ID'].unique()
items = new_ratings['ISBN'].unique()
model = Model(rating_matrix, users, items)

In [7]:
def recommend_items(user_id):
  print(f'{user_id}번 사용자가 좋아하는 책(가장 평점을 높게 준 아이템)')
  
  favorite_books_id = new_ratings.loc[new_ratings['User-ID'] == user_id].sort_values('Book-Rating', ascending=False)['ISBN'][:10]
  favorite_books = rating_books.loc[rating_books['ISBN'].isin(favorite_books_id)]
  display(favorite_books)

  print('추천 아이템 : ')
  recommend_items = model.recommend(user_id)
  recommend_books = rating_books.loc[rating_books['ISBN'].isin(recommend_items)]
  display(recommend_books)

# 가장 많은 평점을 부여한 사용자 확인하기

In [11]:
new_ratings['User-ID'].value_counts().head(1)

11676    6943
Name: User-ID, dtype: int64

11676번 사용자가 총 6943개의 아이템에 평가를 했습니다.

In [17]:
6943 / new_ratings['ISBN'].nunique() * 100

4.633732881283537

전체 제품의 약 4%에 해당하는 책에 점수를 부여한 것입니다.

In [18]:
user_id = 11676
recommend_items(user_id)

11676번 사용자가 좋아하는 책(가장 평점을 높게 준 아이템)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
3608,0373483880,The Macgregors: Serena - Caine (The Macgregors),Nora Roberts,1998,Silhouette,http://images.amazon.com/images/P/0373483880.0...,http://images.amazon.com/images/P/0373483880.0...,http://images.amazon.com/images/P/0373483880.0...
5464,0373484232,Stanislaski Sisters,Nora Roberts,2001,Silhouette,http://images.amazon.com/images/P/0373484232.0...,http://images.amazon.com/images/P/0373484232.0...,http://images.amazon.com/images/P/0373484232.0...
14663,0373483503,Macgregor Brides (Macgregors),Nora Roberts,1997,Silhouette,http://images.amazon.com/images/P/0373483503.0...,http://images.amazon.com/images/P/0373483503.0...,http://images.amazon.com/images/P/0373483503.0...
20709,0373483899,The Macgregors: Alan - Grant (The Macgregors),Nora Roberts,1999,Silhouette,http://images.amazon.com/images/P/0373483899.0...,http://images.amazon.com/images/P/0373483899.0...,http://images.amazon.com/images/P/0373483899.0...
67839,0770428312,The Dionnes,Ellie Tesher,2000,Bantam Books,http://images.amazon.com/images/P/0770428312.0...,http://images.amazon.com/images/P/0770428312.0...,http://images.amazon.com/images/P/0770428312.0...
71003,1585790370,My Cousin Rachel (Common Reader Editions),Daphne Du Maurier,2001,Akadine Press,http://images.amazon.com/images/P/1585790370.0...,http://images.amazon.com/images/P/1585790370.0...,http://images.amazon.com/images/P/1585790370.0...
72358,0373484127,36 Hours Christmas (Silhouette Promo),Mary Lynn Baxter,2000,Silhouette,http://images.amazon.com/images/P/0373484127.0...,http://images.amazon.com/images/P/0373484127.0...,http://images.amazon.com/images/P/0373484127.0...
82058,B0000DYXID,A Kid's Guide To How to Save the Planet,Billy Goodman,0,"iBooks, Inc.",http://images.amazon.com/images/P/B0000DYXID.0...,http://images.amazon.com/images/P/B0000DYXID.0...,http://images.amazon.com/images/P/B0000DYXID.0...
115010,0373482892,Change Of Place - (Movie Tie-In),Tracy Sinclair,1994,Silhouette,http://images.amazon.com/images/P/0373482892.0...,http://images.amazon.com/images/P/0373482892.0...,http://images.amazon.com/images/P/0373482892.0...
130471,0517885042,Ultimate Japanese: Advanced (Living Language U...,SUGURU AKUTSU,1998,Living Language,http://images.amazon.com/images/P/0517885042.0...,http://images.amazon.com/images/P/0517885042.0...,http://images.amazon.com/images/P/0517885042.0...


추천 아이템 : 
피어그룹 계산 중...
피어그룹 계산 완료
추천 완료


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1503,037348285X,Birds Bees And Babies 1994,Nora Roberts,1994,Silhouette,http://images.amazon.com/images/P/037348285X.0...,http://images.amazon.com/images/P/037348285X.0...,http://images.amazon.com/images/P/037348285X.0...
1860,0517077744,Sandra Brown: Three Complete Novels,SANDRA BROWN,1992,Wings,http://images.amazon.com/images/P/0517077744.0...,http://images.amazon.com/images/P/0517077744.0...,http://images.amazon.com/images/P/0517077744.0...
1863,0373510225,Sullivan'S Woman #22 (Nora Roberts : Language ...,Nora Roberts,1992,Silhouette,http://images.amazon.com/images/P/0373510225.0...,http://images.amazon.com/images/P/0373510225.0...,http://images.amazon.com/images/P/0373510225.0...
5317,0399144706,River's End,Nora Roberts,1999,Putnam Publishing Group,http://images.amazon.com/images/P/0399144706.0...,http://images.amazon.com/images/P/0399144706.0...,http://images.amazon.com/images/P/0399144706.0...
5769,0373834284,Rebellion,Nora Roberts,1999,Harlequin,http://images.amazon.com/images/P/0373834284.0...,http://images.amazon.com/images/P/0373834284.0...,http://images.amazon.com/images/P/0373834284.0...
41979,051512947X,Once upon a Dream,Nora Roberts,2000,Jove Books,http://images.amazon.com/images/P/051512947X.0...,http://images.amazon.com/images/P/051512947X.0...,http://images.amazon.com/images/P/051512947X.0...
70480,0373790988,Slippery When Wet: Under the Covers (Harlequin...,Kristin Hardy,2003,Harlequin,http://images.amazon.com/images/P/0373790988.0...,http://images.amazon.com/images/P/0373790988.0...,http://images.amazon.com/images/P/0373790988.0...
95265,0373510020,Law Is A Lady #2 (Language Of Love) (Nora Rob...,Nora Roberts,1992,Silhouette,http://images.amazon.com/images/P/0373510020.0...,http://images.amazon.com/images/P/0373510020.0...,http://images.amazon.com/images/P/0373510020.0...
95269,037351011X,Search For Love,Nora Roberts,1992,Silhouette,http://images.amazon.com/images/P/037351011X.0...,http://images.amazon.com/images/P/037351011X.0...,http://images.amazon.com/images/P/037351011X.0...
95285,0373510381,Blithe Images (Language of Love No. 38),Nora Roberts,1993,Silhouette,http://images.amazon.com/images/P/0373510381.0...,http://images.amazon.com/images/P/0373510381.0...,http://images.amazon.com/images/P/0373510381.0...


6분 30초 정도 소요

# 가장 적은 평점을 부여한 사용자 확인하기

In [19]:
new_ratings['User-ID'].value_counts().tail(1)

276721    1
Name: User-ID, dtype: int64

In [20]:
user_id = 276721
recommend_items(user_id)

276721번 사용자가 좋아하는 책(가장 평점을 높게 준 아이템)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
42169,0590442449,Fourth Grade Rats,Jerry Spinelli,1996,Scholastic,http://images.amazon.com/images/P/0590442449.0...,http://images.amazon.com/images/P/0590442449.0...,http://images.amazon.com/images/P/0590442449.0...


추천 아이템 : 
피어그룹 계산 중...
피어그룹 계산 완료
추천 완료


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
3258,0345308999,Foundation (Foundation Novels (Paperback)),Isaac Asimov,1983,Del Rey Books,http://images.amazon.com/images/P/0345308999.0...,http://images.amazon.com/images/P/0345308999.0...,http://images.amazon.com/images/P/0345308999.0...
5117,039447970X,Centennial,James A. Michener,1974,Random House,http://images.amazon.com/images/P/039447970X.0...,http://images.amazon.com/images/P/039447970X.0...,http://images.amazon.com/images/P/039447970X.0...
12768,0836218663,The Revenge Of The Baby-Sat,Bill Watterson,1991,Andrews McMeel Publishing,http://images.amazon.com/images/P/0836218663.0...,http://images.amazon.com/images/P/0836218663.0...,http://images.amazon.com/images/P/0836218663.0...
18498,0880705760,No Wonder They Call Him Savior (Chronicles of ...,Max Lucado,1986,Word Publishing,http://images.amazon.com/images/P/0880705760.0...,http://images.amazon.com/images/P/0880705760.0...,http://images.amazon.com/images/P/0880705760.0...
21480,0451148428,Hoyles Rules of Games 2/E,Albert H. Morehead,1983,Signet Book,http://images.amazon.com/images/P/0451148428.0...,http://images.amazon.com/images/P/0451148428.0...,http://images.amazon.com/images/P/0451148428.0...
63851,0553293427,Pebble in the Sky (The Empire Novels),Isaac Asimov,1992,Spectra Books,http://images.amazon.com/images/P/0553293427.0...,http://images.amazon.com/images/P/0553293427.0...,http://images.amazon.com/images/P/0553293427.0...
76702,0812096533,Spanish Now: Level 1,Ruth J. Silverstein,1997,Barron's Educational Series,http://images.amazon.com/images/P/0812096533.0...,http://images.amazon.com/images/P/0812096533.0...,http://images.amazon.com/images/P/0812096533.0...
76735,0060006285,For Everything a Season: Simple Musings on Liv...,Philip Gulley,2001,Harper SanFrancisco,http://images.amazon.com/images/P/0060006285.0...,http://images.amazon.com/images/P/0060006285.0...,http://images.amazon.com/images/P/0060006285.0...
76760,0849913519,The Tale Of The Tardy Oxcart,Charles R. Swindoll,1998,W Publishing Group,http://images.amazon.com/images/P/0849913519.0...,http://images.amazon.com/images/P/0849913519.0...,http://images.amazon.com/images/P/0849913519.0...
76987,0191012084,The Revised English Bible,Oxford University Press,1989,Oxford University Press,http://images.amazon.com/images/P/0191012084.0...,http://images.amazon.com/images/P/0191012084.0...,http://images.amazon.com/images/P/0191012084.0...
